In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import openpyxl
#import plotly.express as px

In [7]:
# Read data
#muri
data = pd.read_excel('./metagenetica.xlsx', sheet_name='El_cielo', engine='openpyxl').append(pd.read_excel('./metagenetica.xlsx', sheet_name='Chamela',engine='openpyxl'))

#mateo
#data = pd.read_excel('./Datos_Metagenetica.xlsx', sheet_name='El_cielo').append(pd.read_excel('./Datos_Metagenetica.xlsx', sheet_name='Chamela'))
# replace 0 with NaN
data = data.replace(0, np.nan)
#drop columns que no se van a usar 
data = data.drop(['Database','.id', 'similarity', 'phylum_final', 
                  'class_final', 'subfamily_final', 'tribe_final',
                   'subspecies_final', 'BASE', 'OTU'], axis=1)
data

C:\Users\mateo\AppData\Local\Temp\ipykernel_34200\3248478459.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = pd.read_excel('./Datos_Metagenetica.xlsx', sheet_name='El_cielo').append(pd.read_excel('./Datos_Metagenetica.xlsx', sheet_name='Chamela'))


,Sequence,order_final,family_final,genus_final,species_final
0,aataaacaatataagattttggttattgcctccttcattatcactc...,Coleoptera,Mordellidae,NaN,NaN
1,aataaataatataagtttttgacttcttcctccttctttaacctta...,Coleoptera,Carabidae,Glyptolenus,NaN
2,tttaaacaatataagattttgattgttaccaccttcattaactttc...,Coleoptera,Coccinellidae,NaN,NaN
3,tataaacaatataagattctgacttcttccaccttcattaagatta...,Coleoptera,Mordellidae,NaN,NaN
4,aataaataatataagattttgactacttcctccgtcacttaccctt...,Coleoptera,Nitidulidae,NaN,NaN
...,...,...,...,...,...
1778,aataaataatataagtttttgacttttacctcctgcattaacactt...,Diptera,Tachinidae,Ischyrophaga,NaN
1779,aataaataatataagattttgattattaccaccatcaataattata...,Hymenoptera,Ichneumonidae,ichneuMalaiseNA1,NaN
1780,aataaataacataagattttgattactcccaccttctcttttttta...,Hymenoptera,Ichneumonidae,NaN,NaN
1781,aataaataatataagtttctgacttcttcccccttctttaattctt...,Lepidoptera,Erebidae,Arugisa,NaN


# Balance data

In [8]:
def balance_one_tax_data(df, col_tax_to_balance, tax_to_balance, max_samples) -> pd.DataFrame:
    '''
    df: dataframe with all data to balance
    col_tax_to_balance: column name of the tax to balance
    tax_to_balance: tax to balance
    max_samples: maximum number to save of each tax
    return: balanced dataframe
    '''
    col_index = df.columns.get_loc(col_tax_to_balance)
    if col_index + 1 < len(df.columns):
        next_tax_col = df.columns[col_index + 1]
        vc = df[df[col_tax_to_balance] == tax_to_balance][next_tax_col].value_counts()
        q25 = vc.quantile(0.25)
        selected_values = vc[vc > 4 * q25]
        balanced = pd.DataFrame(columns = df.columns)
        for i in selected_values.index:
            if vc[i] > max_samples:
                balanced = pd.concat((balanced, df[df[next_tax_col] == i].sample(max_samples)))
            else:
                balanced = pd.concat((balanced, df[df[next_tax_col] == i]))
    return balanced

print('Original data')
print(data.value_counts('family_final'))
print('Balanced data')
balance_one_tax_data(data, 'order_final', 'Diptera', 50).value_counts('family_final')

Original data
family_final
Erebidae           236
Tachinidae         200
Phoridae           162
Ichneumonidae      154
Cecidomyiidae      143
                  ... 
Phacopteronidae      1
Perilampidae         1
Derbidae             1
Disteniidae          1
f__Triozidae         1
Length: 216, dtype: int64
Balanced data


family_final
Cecidomyiidae      50
Ceratopogonidae    50
Sciaridae          50
Phoridae           50
Tachinidae         50
Sarcophagidae      33
Drosophilidae      32
Chironomidae       31
Dolichopodidae     30
Mycetophilidae     25
Muscidae           24
Syrphidae          24
Chloropidae        22
Stratiomyidae      20
Sphaeroceridae     19
Pipunculidae       15
Lauxaniidae        14
Tabanidae          14
Limoniidae         12
Psychodidae        10
dtype: int64

---
# Encoding & Concatenation

In [3]:
data['Sequence'] = data['Sequence'].apply(lambda x: x.upper())

In [4]:
def sequence_encoding(sequence):
    mapping = {"A": 0, "C": 1, "G": 2, "T": 3}
    encoded_sequence = [mapping[i] for i in sequence]
    return np.eye(4)[encoded_sequence]

In [5]:
elem0 = data['Sequence'].iloc[0]
elem1 = data['Sequence'].iloc[1]
elem0

'AATAAACAATATAAGATTTTGGTTATTGCCTCCTTCATTATCACTCCTTTTAATAAGAAGAATCGTAGAAACCGGTGCAGGTACAGGTTGAACAGTGTACCCCCCGCTGTCATCCAATATTGCCCACAGAGGTGCTTCAGTTGATTTAGCTATTTTTAGACTACATTTAGCTGGTATTTCTTCTATTTTAGGAGCAATTAATTTTATTTCTACAATAATTAATATACGACCCGCAGGAATAACCTTTGACCGAATACCCTTATTTGTCTGAGCTATTGCTATTACTGCCGTACTTCTACTATTATCTCTTCCTGTCTTAGCTGGAGCAATTACTATATTATTAACTGATCGAAATTTAAATACTACCTTTTTTGATCCCGCCGGAGGAGGAGATCCAATCTTATATCAACATCTCTTT'

In [6]:
enc0 = sequence_encoding(elem0)
enc1 = sequence_encoding(elem1)
enc0

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [7]:
enc1.shape

(418, 4)

In [8]:
def long_sequence(seq1, seq2):
    return np.concatenate((seq1, seq2), axis=1)

In [9]:
long_seq = long_sequence(enc0,enc1)
long_seq.shape
#type(long_seq)

(418, 8)

In [10]:
long_seq

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [11]:
def deep_sequence(seq1, seq2):
    s1 = seq1[np.newaxis, :, :]
    s2 = seq2[np.newaxis, :, :]
    return np.concatenate((s1, s2), axis=0)

In [12]:
deep_seq = deep_sequence(enc0,enc1)
deep_seq.shape

(2, 418, 4)

In [13]:
deep_seq

array([[[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        ...,
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.]],

       [[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        ...,
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.]]])

---
# Combinaciones

In [14]:
datos = {'Col1': ['C', 'A', 'R','M', 'E', 'N'],
        'Col2': ['S','S','M','S','M','M']}

df = pd.DataFrame(datos)
df

,Col1,Col2
0,C,S
1,A,S
2,R,M
3,M,S
4,E,M
5,N,M


In [15]:
import itertools

def combination_list(dataframe,column):
    sequences = dataframe[column].tolist()
    combinations = list(itertools.combinations(sequences, 2))
    return combinations

In [16]:
combinations = combination_list(df,'Col1')
combinations

[('C', 'A'),
 ('C', 'R'),
 ('C', 'M'),
 ('C', 'E'),
 ('C', 'N'),
 ('A', 'R'),
 ('A', 'M'),
 ('A', 'E'),
 ('A', 'N'),
 ('R', 'M'),
 ('R', 'E'),
 ('R', 'N'),
 ('M', 'E'),
 ('M', 'N'),
 ('E', 'N')]

In [17]:
def combination_matrix(df,sequence_column,tax_column):
    combinaciones = []
    for sequence1, sequence2 in combination_list(df,sequence_column):
        z = zip(df[df[sequence_column] == sequence1][tax_column], df[df[sequence_column] == sequence2][tax_column])
        for clase1, clase2 in z:
            combinaciones.append([sequence1, sequence2, clase1, clase2])

    df_combinaciones = pd.DataFrame(combinaciones, columns=['Sequence1', 'Sequence2', 'Tax1', 'Tax2'])
    return df_combinaciones

In [18]:
nuevo_df = combination_matrix(df,'Col1','Col2')
nuevo_df

,Sequence1,Sequence2,Tax1,Tax2
0,C,A,S,S
1,C,R,S,M
2,C,M,S,S
3,C,E,S,M
4,C,N,S,M
5,A,R,S,M
6,A,M,S,S
7,A,E,S,M
8,A,N,S,M
9,R,M,M,S


In [19]:
p = data.head(1000)
p

,Sequence,order_final,family_final,genus_final,species_final,Unnamed: 13
0,AATAAACAATATAAGATTTTGGTTATTGCCTCCTTCATTATCACTC...,Coleoptera,Mordellidae,NaN,NaN,NaN
1,AATAAATAATATAAGTTTTTGACTTCTTCCTCCTTCTTTAACCTTA...,Coleoptera,Carabidae,Glyptolenus,NaN,NaN
2,TTTAAACAATATAAGATTTTGATTGTTACCACCTTCATTAACTTTC...,Coleoptera,Coccinellidae,NaN,NaN,NaN
3,TATAAACAATATAAGATTCTGACTTCTTCCACCTTCATTAAGATTA...,Coleoptera,Mordellidae,NaN,NaN,NaN
4,AATAAATAATATAAGATTTTGACTACTTCCTCCGTCACTTACCCTT...,Coleoptera,Nitidulidae,NaN,NaN,NaN
...,...,...,...,...,...,...
995,AATAAATAATATGAGATTTTGGTTGCTCCCGCCTTCATTAACTTTA...,Hemiptera,Cicadellidae,NaN,NaN,NaN
996,AATAAATAATATAAGATTTTGATTATTACCACCATCTTTAATTACC...,Hymenoptera,Ichneumonidae,Hyposoter,NaN,NaN
997,AATAAATAATATAAGATTTTGGTTATTAATTCCTTCTTTAATACTC...,Hymenoptera,Braconidae,Aleiodes,NaN,NaN
998,AATAAATAATATAAGATTTTGATTATTACCCCCTTCTTTAATAATT...,Hymenoptera,Ichneumonidae,Hyposoter,NaN,NaN


In [20]:
comb = combination_matrix(p,'Sequence',"order_final")
comb

,Sequence1,Sequence2,Tax1,Tax2
0,AATAAACAATATAAGATTTTGGTTATTGCCTCCTTCATTATCACTC...,AATAAATAATATAAGTTTTTGACTTCTTCCTCCTTCTTTAACCTTA...,Coleoptera,Coleoptera
1,AATAAACAATATAAGATTTTGGTTATTGCCTCCTTCATTATCACTC...,TTTAAACAATATAAGATTTTGATTGTTACCACCTTCATTAACTTTC...,Coleoptera,Coleoptera
2,AATAAACAATATAAGATTTTGGTTATTGCCTCCTTCATTATCACTC...,TATAAACAATATAAGATTCTGACTTCTTCCACCTTCATTAAGATTA...,Coleoptera,Coleoptera
3,AATAAACAATATAAGATTTTGGTTATTGCCTCCTTCATTATCACTC...,AATAAATAATATAAGATTTTGACTACTTCCTCCGTCACTTACCCTT...,Coleoptera,Coleoptera
4,AATAAACAATATAAGATTTTGGTTATTGCCTCCTTCATTATCACTC...,AATAAACAATATAAGATTTTGACTTTTACCTCCTTCTTTAACTTTA...,Coleoptera,Coleoptera
...,...,...,...,...
499495,AATAAATAATATAAGATTTTGATTATTACCACCATCTTTAATTACC...,AATAAATAATATAAGATTTTGATTATTACCCCCTTCTTTAATAATT...,Hymenoptera,Hymenoptera
499496,AATAAATAATATAAGATTTTGATTATTACCACCATCTTTAATTACC...,AATAAATAACATAAGTTTTTGATTATTACCTCCATCATTAATTTTA...,Hymenoptera,Lepidoptera
499497,AATAAATAATATAAGATTTTGGTTATTAATTCCTTCTTTAATACTC...,AATAAATAATATAAGATTTTGATTATTACCCCCTTCTTTAATAATT...,Hymenoptera,Hymenoptera
499498,AATAAATAATATAAGATTTTGGTTATTAATTCCTTCTTTAATACTC...,AATAAATAACATAAGTTTTTGATTATTACCTCCATCATTAATTTTA...,Hymenoptera,Lepidoptera


In [21]:
# 3:01
# 3:01.05

In [22]:
def tax_comparison(dataframe, tax1, tax2):
    dataframe['Same'] = dataframe[tax1] == dataframe[tax2]
    return dataframe

In [23]:
input_matrix = tax_comparison(comb,'Tax1','Tax2')
input_matrix

,Sequence1,Sequence2,Tax1,Tax2,Same
0,AATAAACAATATAAGATTTTGGTTATTGCCTCCTTCATTATCACTC...,AATAAATAATATAAGTTTTTGACTTCTTCCTCCTTCTTTAACCTTA...,Coleoptera,Coleoptera,True
1,AATAAACAATATAAGATTTTGGTTATTGCCTCCTTCATTATCACTC...,TTTAAACAATATAAGATTTTGATTGTTACCACCTTCATTAACTTTC...,Coleoptera,Coleoptera,True
2,AATAAACAATATAAGATTTTGGTTATTGCCTCCTTCATTATCACTC...,TATAAACAATATAAGATTCTGACTTCTTCCACCTTCATTAAGATTA...,Coleoptera,Coleoptera,True
3,AATAAACAATATAAGATTTTGGTTATTGCCTCCTTCATTATCACTC...,AATAAATAATATAAGATTTTGACTACTTCCTCCGTCACTTACCCTT...,Coleoptera,Coleoptera,True
4,AATAAACAATATAAGATTTTGGTTATTGCCTCCTTCATTATCACTC...,AATAAACAATATAAGATTTTGACTTTTACCTCCTTCTTTAACTTTA...,Coleoptera,Coleoptera,True
...,...,...,...,...,...
499495,AATAAATAATATAAGATTTTGATTATTACCACCATCTTTAATTACC...,AATAAATAATATAAGATTTTGATTATTACCCCCTTCTTTAATAATT...,Hymenoptera,Hymenoptera,True
499496,AATAAATAATATAAGATTTTGATTATTACCACCATCTTTAATTACC...,AATAAATAACATAAGTTTTTGATTATTACCTCCATCATTAATTTTA...,Hymenoptera,Lepidoptera,False
499497,AATAAATAATATAAGATTTTGGTTATTAATTCCTTCTTTAATACTC...,AATAAATAATATAAGATTTTGATTATTACCCCCTTCTTTAATAATT...,Hymenoptera,Hymenoptera,True
499498,AATAAATAATATAAGATTTTGGTTATTAATTCCTTCTTTAATACTC...,AATAAATAACATAAGTTTTTGATTATTACCTCCATCATTAATTTTA...,Hymenoptera,Lepidoptera,False


---
# Final Matrix

In [24]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def final_matrix(data,concat_type):
    
    #Pasamos a letras mayúsculas las cadenas
    data['Sequence'] = data['Sequence'].apply(lambda x: x.upper())
    #Llamamos a la función para filtrar los datos
    
    #Llamamos a la función para realizar todas las combinaciones
    combinations = combination_matrix(data,'Sequence',"order_final")
    #Llamamos a la función para determinar si los Taxones son iguales
    DNA_matrix = tax_comparison(combinations,'Tax1','Tax2')
    #Pasamos los valores boolean a integer
    DNA_matrix['Same'] = DNA_matrix['Same'].astype(int)
    
    DNA_matrix['Paired_seq'] = ''  # Creamos una columna vacía para almacenar los resultados
    for index, row in DNA_matrix.iterrows():
        sequence1 = row['Sequence1']
        sequence2 = row['Sequence2']
        encoding1 = sequence_encoding(sequence1)
        encoding2 = sequence_encoding(sequence2)
        vectors_padded = pad_sequences([encoding1, encoding2], padding='post')
        paired_sequences = concat_type(vectors_padded[0], vectors_padded[0])
        DNA_matrix.at[index, 'Paired_seq'] = paired_sequences
    return DNA_matrix

In [25]:
DNA_matrix = final_matrix(p,deep_sequence)
DNA_matrix

/opt/anaconda3/envs/deep_learning/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Sequence1,Sequence2,Tax1,Tax2,Same,Paired_seq
0,AATAAACAATATAAGATTTTGGTTATTGCCTCCTTCATTATCACTC...,AATAAATAATATAAGTTTTTGACTTCTTCCTCCTTCTTTAACCTTA...,Coleoptera,Coleoptera,1,"[[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [1..."
1,AATAAACAATATAAGATTTTGGTTATTGCCTCCTTCATTATCACTC...,TTTAAACAATATAAGATTTTGATTGTTACCACCTTCATTAACTTTC...,Coleoptera,Coleoptera,1,"[[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [1..."
2,AATAAACAATATAAGATTTTGGTTATTGCCTCCTTCATTATCACTC...,TATAAACAATATAAGATTCTGACTTCTTCCACCTTCATTAAGATTA...,Coleoptera,Coleoptera,1,"[[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [1..."
3,AATAAACAATATAAGATTTTGGTTATTGCCTCCTTCATTATCACTC...,AATAAATAATATAAGATTTTGACTACTTCCTCCGTCACTTACCCTT...,Coleoptera,Coleoptera,1,"[[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [1..."
4,AATAAACAATATAAGATTTTGGTTATTGCCTCCTTCATTATCACTC...,AATAAACAATATAAGATTTTGACTTTTACCTCCTTCTTTAACTTTA...,Coleoptera,Coleoptera,1,"[[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [1..."
...,...,...,...,...,...,...
499495,AATAAATAATATAAGATTTTGATTATTACCACCATCTTTAATTACC...,AATAAATAATATAAGATTTTGATTATTACCCCCTTCTTTAATAATT...,Hymenoptera,Hymenoptera,1,"[[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [1..."
499496,AATAAATAATATAAGATTTTGATTATTACCACCATCTTTAATTACC...,AATAAATAACATAAGTTTTTGATTATTACCTCCATCATTAATTTTA...,Hymenoptera,Lepidoptera,0,"[[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [1..."
499497,AATAAATAATATAAGATTTTGGTTATTAATTCCTTCTTTAATACTC...,AATAAATAATATAAGATTTTGATTATTACCCCCTTCTTTAATAATT...,Hymenoptera,Hymenoptera,1,"[[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [1..."
499498,AATAAATAATATAAGATTTTGGTTATTAATTCCTTCTTTAATACTC...,AATAAATAACATAAGTTTTTGATTATTACCTCCATCATTAATTTTA...,Hymenoptera,Lepidoptera,0,"[[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [1..."


In [26]:
DNA_matrix.dtypes

Sequence1     object
Sequence2     object
Tax1          object
Tax2          object
Same           int64
Paired_seq    object
dtype: object

---
# Data preparation for CNN

In [40]:
from sklearn.model_selection import train_test_split

# Obtenemos las secuencias de ADN en una variable 'X' y las etiquetas en una variable 'y'
DNA = DNA_matrix.iloc[:,5]  # Columna 5: pares de secuencias
labels = DNA_matrix.iloc[:,4]  # Columna 4: etiquetas
#X

In [42]:
# Divide los datos en conjuntos de entrenamiento y prueba (80% para entrenamiento, 20% para prueba)
DNA_train, DNA_test, labels_train, labels_test = train_test_split(
    DNA, labels, test_size=0.20, random_state=42)

print("Forma de X_train:", DNA_train.shape)
print("Forma de y_train:", labels_train.shape)

print("Forma de X_test:", DNA_test.shape)
print("Forma de y_test:", labels_test.shape)

Forma de X_train: (399600,)
Forma de y_train: (399600,)
Forma de X_test: (99900,)
Forma de y_test: (99900,)
